# AspC insertion

In [1]:
%run 1_Objective_function.ipynb

EX_asp__L_e EX_asp__L_e: asp__L_e -->  L-Aspartate exchange
ASPtex ASPtex: asp__L_e <=> asp__L_p L-aspartate transport via diffusion (extracellular to periplasm)
0.87699721425716
0.87392
4.596027557488181


In [12]:
%pip install cameo

Note: you may need to restart the kernel to use updated packages.


El sistema no puede encontrar la ruta especificada.


In [15]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite 

Firstly, we will recover the reactions that produce L-aspartate from the model.

In [16]:
from cameo.strain_design import pathway_prediction
predictor = pathway_prediction.PathwayPredictor(model_asp)
pathways = predictor.run(product="asp__L_c", max_predictions=4)

ImportError: cannot import name 'find_gene_knockout_reactions' from 'cobra.manipulation.delete' (c:\Users\Candela\anaconda3\lib\site-packages\cobra\manipulation\delete.py)

Then, we will add a new reaction that produces L-aspartate from OAA transamination using Pseudomonas aeruginosa's AspC (PaeAspDH). This reaction will be added to the model as a heterologous reaction.



In [18]:
PaeAspDH_reaction = Reaction('R_PaeAspDH')

new_reaction.add_metabolites({
                            model.metabolites.get_by_id("glu__L_c"): -1,
                            model.metabolites.get_by_id("oaa_c"): -1,
                            model.metabolites.get_by_id("akg_c"): 1, # our newly created metabolites
                            model.metabolites.get_by_id("asp__L_c"): 1
                             })

model_asp.add_reactions([PaeAspDH_reaction])

NameError: name 'new_reaction' is not defined